In [1]:
import sys
sys.path.append(str('/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/'))
import pandas as pd

from repo.MyConfigParser import MyConfigParser

from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts
from repo.FileGenerator import *
from repo.QueryingData import *

In [2]:

config_in_path = "/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/emea/test/config ups emea.ini"
config_file = MyConfigParser(config_in_path)
config_dict = config_file.GetDict()

region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

training_query = config_dict["PROPHETSETTING"]["training_query"]
_input = config_dict["PROPHETSETTING"]["input"]
_output = config_dict["PROPHETSETTING"]["output"]

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]


today = datetime.date.today().strftime("%m.%d.%Y") #'03.10.2023' #
print(today)
base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]


06.12.2024


In [3]:
training_query

"SELECT region, market, account_type, funnel, Date, sum(cost) AS X, sum(conversion_quantity) AS y, sum(dashboard_revenue) AS revenue FROM `ups-analytics.bx_forecast_optimization.Region_market_daily_data_mar_2023_upd` WHERE market NOT IN ('Austria', 'Switzerland') AND region = 'Europe' AND DATE_TRUNC(date, MONTH) != '2022-11-01' GROUP BY region, market, account_type, funnel, Date ORDER BY region, market, account_type, funnel, Date"

In [4]:
daily_df = GetBQData(training_query, _input, _output) # import data from BQ


In [5]:
small_market_str = ['SaudiArabia', 'UAE','Turkey','Ireland','Sweden','Netherlands','Belgium','Czech','Spain']

daily_df['Identifier'] = np.where(daily_df['market'].isin(small_market_str), 'S', 'L')


In [6]:

breakdown_columns = [config_dict['CLIENTINFO'][i] for i in sorted([key for key in config_dict['CLIENTINFO'] if key.startswith('breakdown')]) if config_dict['CLIENTINFO'][i] != None]
breakdown_columns = breakdown_columns + ['Identifier']
breakdown_columns
distinct_cuts = daily_df[breakdown_columns].drop_duplicates().reset_index(drop = True)



In [7]:
idx = distinct_cuts.astype(str).apply(lambda row: ' '.join(row), axis=1)

In [8]:
date_index = pd.date_range(daily_df.ds.min(), daily_df.ds.max()) 
date_index

DatetimeIndex(['2017-08-10', '2017-08-11', '2017-08-12', '2017-08-13',
               '2017-08-14', '2017-08-15', '2017-08-16', '2017-08-17',
               '2017-08-18', '2017-08-19',
               ...
               '2024-06-02', '2024-06-03', '2024-06-04', '2024-06-05',
               '2024-06-06', '2024-06-07', '2024-06-08', '2024-06-09',
               '2024-06-10', '2024-06-11'],
              dtype='datetime64[ns]', length=2498, freq='D')

In [9]:
multiple_index = pd.MultiIndex.from_product([date_index, idx], names = ['ds','idx'])
multiple_index

MultiIndex([('2017-08-10',            'Belgium Bing Ads Brand S'),
            ('2017-08-10',       'Belgium Bing Ads Competitor S'),
            ('2017-08-10',          'Belgium Bing Ads Generic S'),
            ('2017-08-10',      'Belgium Google AdWords Brand S'),
            ('2017-08-10', 'Belgium Google AdWords Competitor S'),
            ('2017-08-10',    'Belgium Google AdWords Generic S'),
            ('2017-08-10',              'Czech Bing Ads Brand S'),
            ('2017-08-10',         'Czech Bing Ads Competitor S'),
            ('2017-08-10',            'Czech Bing Ads Generic S'),
            ('2017-08-10',        'Czech Google AdWords Brand S'),
            ...
            ('2024-06-11',     'Turkey Google AdWords Generic S'),
            ('2024-06-11',          'UAE Google AdWords Brand S'),
            ('2024-06-11',     'UAE Google AdWords Competitor S'),
            ('2024-06-11',        'UAE Google AdWords Generic S'),
            ('2024-06-11',                 'UK

In [10]:
daily_df['temp_idx'] = daily_df[breakdown_columns].astype(str).apply(lambda row: ' '.join(row), axis=1)

In [11]:
daily_df = daily_df.set_index(['ds', 'temp_idx']).reindex(multiple_index)

In [12]:
daily_df

region   market  \
ds         idx                                                    
2017-08-10 Belgium Bing Ads Brand S                NaN      NaN   
           Belgium Bing Ads Competitor S           NaN      NaN   
           Belgium Bing Ads Generic S              NaN      NaN   
           Belgium Google AdWords Brand S       Europe  Belgium   
           Belgium Google AdWords Competitor S     NaN      NaN   
...                                                ...      ...   
2024-06-11 UK Bing Ads Competitor L             Europe       UK   
           UK Bing Ads Generic L                Europe       UK   
           UK Google AdWords Brand L            Europe       UK   
           UK Google AdWords Competitor L       Europe       UK   
           UK Google AdWords Generic L          Europe       UK   

                                                  account_type      funnel  \
ds         idx                                                               
2017-08-10 Belgium Bing Ads Brand S                        NaN         NaN   
           Belgium Bing Ads Competitor S                   NaN         NaN   
           Belgium Bing Ads Generic S                      NaN         NaN   
           Belgium Google AdWords Brand S       Google AdWords       Brand   
           Belgium Google AdWords Competitor S             NaN         NaN   
...                                                        ...         ...   
2024-06-11 UK Bing Ads Competitor L                   Bing Ads  Competitor   
           UK Bing Ads Generic L                      Bing Ads     Generic   
           UK Google AdWords Brand L            Google AdWords       Brand   
           UK Google AdWords Competitor L       Google AdWords  Competitor   
           UK Google AdWords Generic L          Google AdWords     Generic   

                                                        X      y  \
ds         idx                                                     
2017-08-10 Belgium Bing Ads Brand S                   NaN    NaN   
           Belgium Bing Ads Competitor S              NaN    NaN   
           Belgium Bing Ads Generic S                 NaN    NaN   
           Belgium Google AdWords Brand S         10.0600   40.0   
           Belgium Google AdWords Competitor S        NaN    NaN   
...                                                   ...    ...   
2024-06-11 UK Bing Ads Competitor L                5.5800    1.0   
           UK Bing Ads Generic L                   0.0000    2.0   
           UK Google AdWords Brand L            2369.8501  606.0   
           UK Google AdWords Competitor L         53.2900    9.0   
           UK Google AdWords Generic L            12.3900    4.0   

                                                     revenue Identifier  
ds         idx                                                           
2017-08-10 Belgium Bing Ads Brand S                      NaN        NaN  
           Belgium Bing Ads Competitor S                 NaN        NaN  
           Belgium Bing Ads Generic S                    NaN        NaN  
           Belgium Google AdWords Brand S         757.635985          S  
           Belgium Google AdWords Competitor S           NaN        NaN  
...                                                      ...        ...  
2024-06-11 UK Bing Ads Competitor L                45.895000          L  
           UK Bing Ads Generic L                   28.193600          L  
           UK Google AdWords Brand L            15927.690310          L  
           UK Google AdWords Competitor L         269.627611          L  
           UK Google AdWords Generic L            151.513798          L  

[202338 rows x 8 columns]

In [13]:
new_df = daily_df.copy()
#new_df.reset_index()

In [14]:
new_df

region   market  \
ds         idx                                                    
2017-08-10 Belgium Bing Ads Brand S                NaN      NaN   
           Belgium Bing Ads Competitor S           NaN      NaN   
           Belgium Bing Ads Generic S              NaN      NaN   
           Belgium Google AdWords Brand S       Europe  Belgium   
           Belgium Google AdWords Competitor S     NaN      NaN   
...                                                ...      ...   
2024-06-11 UK Bing Ads Competitor L             Europe       UK   
           UK Bing Ads Generic L                Europe       UK   
           UK Google AdWords Brand L            Europe       UK   
           UK Google AdWords Competitor L       Europe       UK   
           UK Google AdWords Generic L          Europe       UK   

                                                  account_type      funnel  \
ds         idx                                                               
2017-08-10 Belgium Bing Ads Brand S                        NaN         NaN   
           Belgium Bing Ads Competitor S                   NaN         NaN   
           Belgium Bing Ads Generic S                      NaN         NaN   
           Belgium Google AdWords Brand S       Google AdWords       Brand   
           Belgium Google AdWords Competitor S             NaN         NaN   
...                                                        ...         ...   
2024-06-11 UK Bing Ads Competitor L                   Bing Ads  Competitor   
           UK Bing Ads Generic L                      Bing Ads     Generic   
           UK Google AdWords Brand L            Google AdWords       Brand   
           UK Google AdWords Competitor L       Google AdWords  Competitor   
           UK Google AdWords Generic L          Google AdWords     Generic   

                                                        X      y  \
ds         idx                                                     
2017-08-10 Belgium Bing Ads Brand S                   NaN    NaN   
           Belgium Bing Ads Competitor S              NaN    NaN   
           Belgium Bing Ads Generic S                 NaN    NaN   
           Belgium Google AdWords Brand S         10.0600   40.0   
           Belgium Google AdWords Competitor S        NaN    NaN   
...                                                   ...    ...   
2024-06-11 UK Bing Ads Competitor L                5.5800    1.0   
           UK Bing Ads Generic L                   0.0000    2.0   
           UK Google AdWords Brand L            2369.8501  606.0   
           UK Google AdWords Competitor L         53.2900    9.0   
           UK Google AdWords Generic L            12.3900    4.0   

                                                     revenue Identifier  
ds         idx                                                           
2017-08-10 Belgium Bing Ads Brand S                      NaN        NaN  
           Belgium Bing Ads Competitor S                 NaN        NaN  
           Belgium Bing Ads Generic S                    NaN        NaN  
           Belgium Google AdWords Brand S         757.635985          S  
           Belgium Google AdWords Competitor S           NaN        NaN  
...                                                      ...        ...  
2024-06-11 UK Bing Ads Competitor L                45.895000          L  
           UK Bing Ads Generic L                   28.193600          L  
           UK Google AdWords Brand L            15927.690310          L  
           UK Google AdWords Competitor L         269.627611          L  
           UK Google AdWords Generic L            151.513798          L  

[202338 rows x 8 columns]

In [18]:
new_df = new_df.reset_index()

In [19]:
new_df

,ds,idx,region,market,account_type,funnel,X,y,revenue,Identifier
0,2017-08-10,Belgium Bing Ads Brand S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-10,Belgium Bing Ads Competitor S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-10,Belgium Bing Ads Generic S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-10,Belgium Google AdWords Brand S,Europe,Belgium,Google AdWords,Brand,10.0600,40.0,757.635985,S
4,2017-08-10,Belgium Google AdWords Competitor S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
202333,2024-06-11,UK Bing Ads Competitor L,Europe,UK,Bing Ads,Competitor,5.5800,1.0,45.895000,L
202334,2024-06-11,UK Bing Ads Generic L,Europe,UK,Bing Ads,Generic,0.0000,2.0,28.193600,L
202335,2024-06-11,UK Google AdWords Brand L,Europe,UK,Google AdWords,Brand,2369.8501,606.0,15927.690310,L
202336,2024-06-11,UK Google AdWords Competitor L,Europe,UK,Google AdWords,Competitor,53.2900,9.0,269.627611,L


In [ ]:
updated_df[breakdown_columns] = updated_df.idx.str.split(expand = True)

In [21]:
M,A,F,I = new_df.idx.str.split(expand = True)

ValueError: too many values to unpack (expected 4)